In [3]:
import pandas as pd
import sqlite3
import os

# Step 2: Define file paths
processed_path = '../data/processed/cleaned_marketing_data.csv'
db_path = '../data/marketing_dashboard.db'
reports_dir = '../reports/'

# Ensure directories exist
os.makedirs('../reports', exist_ok=True)

# Check if cleaned dataset exists
if not os.path.exists(processed_path):
    raise FileNotFoundError(f"The cleaned dataset was not found at {processed_path}. Please run 01_data_cleaning.ipynb first.")

# Step 3: Load cleaned dataset
df = pd.read_csv(processed_path)
print("Loaded cleaned dataset with shape:", df.shape)

# Step 4: Connect to SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
print(f"Connected to database: {db_path}")

# Step 5: Create SQL table and load data
table_name = 'campaign_performance'
df.to_sql(table_name, conn, if_exists='replace', index=False)
print(f"Loaded data into table: {table_name}")

# Step 6: Run sample analytical queries
queries = {
    'Top Campaign Types by ROI': f"""
        SELECT Campaign_Type, ROUND(AVG(ROI), 2) AS Avg_ROI
        FROM {table_name}
        GROUP BY Campaign_Type
        ORDER BY Avg_ROI DESC
        LIMIT 5;
    """,
    'Top Campaigns by Conversion Rate': f"""
        SELECT Company, Campaign_Type, ROUND(AVG(Conversion_Rate)*100, 2) AS Avg_Conversion_Rate
        FROM {table_name}
        GROUP BY Company, Campaign_Type
        ORDER BY Avg_Conversion_Rate DESC
        LIMIT 5;
    """,
    'Overall Channel Performance': f"""
        SELECT Channel_Used, 
               ROUND(AVG(CTR)*100, 2) AS Avg_CTR, 
               ROUND(AVG(CPC), 2) AS Avg_CPC,
               ROUND(AVG(ROI), 2) AS Avg_ROI
        FROM {table_name}
        GROUP BY Channel_Used
        ORDER BY Avg_ROI DESC;
    """,
}

# Step 7: Execute and display results
for name, query in queries.items():
    print(f"\n--- {name} ---")
    result = pd.read_sql_query(query, conn)
    print(result.head())

# Step 8: Close connection
conn.close()
print(f"\nDatabase saved at {db_path} and connection closed.")
print(f"Reports available in {reports_dir}")

Loaded cleaned dataset with shape: (200000, 19)
Connected to database: ../data/marketing_dashboard.db
Loaded data into table: campaign_performance

--- Top Campaign Types by ROI ---
  Campaign_Type  Avg_ROI
0        Search     5.01
1    Influencer     5.01
2       Display     5.01
3  Social Media     4.99
4         Email     4.99

--- Top Campaigns by Conversion Rate ---
               Company Campaign_Type  Avg_Conversion_Rate
0   Datatech Solutions    Influencer                 8.11
1  Innovate Industries  Social Media                 8.11
2             Techcorp  Social Media                 8.11
3   Datatech Solutions       Display                 8.07
4  Innovate Industries    Influencer                 8.05

--- Overall Channel Performance ---
  Channel_Used  Avg_CTR  Avg_CPC  Avg_ROI
0     Facebook    14.05      0.0     5.02
1      Website    14.10      0.0     5.01
2   Google Ads    13.92      0.0     5.00
3        Email    14.05      0.0     5.00
4      YouTube    14.12      0.